# TMA4320 - LU-faktorisering med pivotering

Python-program for å gjøre LU-faktorisering med pivotering.
$$
    PA = LU
$$
der $P$ er en permutasjonsmatrise, $L$ er en nedretriangulær matrise med 1'ere på diagonalen, og $U$ er en øvre-triangular matrise. Med *representasjon av matriser* tenker vi oss følgende: Permutasjonsmatrisen $P$ er representert av en vektor $\mathtt{P}$ slik at rad $k$ i matrisen $P$ er enhetsvektor $e_{\mathtt{P}_k}$. Illustrerer med et eksempel
$$
\mathtt{P}=
\left[
\begin{array}{r} 3 \\ 1 \\ 2 \end{array}
\right]\quad\Rightarrow\quad
P=\left[
\begin{array}{ccc}
0 & 0 & 1 \\ 1 & 0 & 0 \\ 0 & 1 & 0
\end{array}
\right]
$$

Antar at Python-funksjonen tar et todimensjonalt array $\mathtt{A}$ som input og returnerer
en *overskrevet* $\mathtt{A}$ som inneholder $L$ og $U$ i følgende forstand ved retur:
$$
\begin{array}{ll}
\mathtt{A}[\mathtt{P}[i],j] = L_{ij} & \text{for}\ i<j \\
\mathtt{A}[\mathtt{P}[i],j] = U_{ij} & \text{for}\ i\geq j
\end{array}
$$
At $L$ har 1 på diagonalen er alltid tilfelle så diagonalen til $L$ trenger ikke å lagres. De øvrige elementene i $L$ og $U$ er null og trenger ikke å lagres. 

**Kommentar:** I praksis fins selvsagt ferdige rutiner både i Python-biblioteker og andre steder for å løse ligningsystemer på en supereffektiv måte. I praksis er det også ofte slik at mange elementer i koeffisientmatrisen $A$ er null og det kan utnyttes på ulike vis. Løseren under er ganske generell. Uansett om programvaren her er hyllevare så er det en god erfaring å én gang ha skrevet et slikt program selv, slik at man forstår hvordan det fungerer, hvilke feil som kan oppstå osv, og det blir enklere å skjønne blant annet feilanalyse.


In [45]:
import numpy as np

In [46]:
def myLU(A):
    LU = A.copy()
    n, m = A.shape
    P = list(range(n))

    
    for k in range (n-1):
        pivot = np.argmax(abs(LU[P[k:], k]))
        pivot += k
        
        P[k], P[pivot] = P[pivot], P[k]
        
        mults = LU[P[k+1:], k]/LU[P[k], k]
        LU[P[k+1:], k+1:] =  LU[P[k+1:], k+1:] - np. outer(mults, LU[P[k], k+1:])
        LU[P[k+1:], k] = mults
        
    return LU, P

    

In [47]:
def forward_subs(LU, P, b):
    n, m = LU.shape
    pb = b[P]
    c = np. zeros(n)
    c[0] = pb[0]
    for k in range(1,n):
        c[k] = pb[k] - LU[P[k],0:k] @ c[0:k]
    return c
        
    

In [48]:
def backward_subs(LU, P, c):
    n, m = LU.shape
    x = np.zeros(n)
    
    x[n-1] = c[n-1]/LU[P[n-1], n-1]
    
    for k in range(n-1, 0, -1):
        x[k-1] = (c[k-1] -(LU[P[k-1],k: ]) @ x[k:])/LU[P[k-1], k-1]
    return x
                

In [49]:
def getAb():
    A=np.array([[0.3050, 0.5399, 0.9831, 0.4039, 0.1962],
                [0.2563, -0.1986, 0.7903, 0.6807, 0.5544],
                [0.7746, 0.6253, -0.1458, 0.1704,  0.5167],
                [0.4406, 0.9256, 0.4361, -0.2254, 0.7784],
                [0.4568, 0.2108, 0.6006, 0.3677, -0.8922]])
    b=np.array([0.9876,-1.231,0.0987,-0.5544,0.7712])
    return A,b
    
A, b = getAb()

print('A =\n', A)
print("\n\n b = \n", b)

LU, p = myLU(A)

print("\n\n LU = \n", LU)
print("\n\n P = \n", p)

c = forward_subs(LU, p, b)

x = backward_subs(LU, p, c)

print("\n\n c = \n", c)

print( "x[4]", x[4])

A =
 [[ 0.305   0.5399  0.9831  0.4039  0.1962]
 [ 0.2563 -0.1986  0.7903  0.6807  0.5544]
 [ 0.7746  0.6253 -0.1458  0.1704  0.5167]
 [ 0.4406  0.9256  0.4361 -0.2254  0.7784]
 [ 0.4568  0.2108  0.6006  0.3677 -0.8922]]


 b = 
 [ 0.9876 -1.231   0.0987 -0.5544  0.7712]


 LU = 
 [[ 0.39375161  0.5153099   0.64002748  0.2501014  -0.72295451]
 [ 0.33088045 -0.71149847  1.20783317  0.3949841   0.72815225]
 [ 0.7746      0.6253     -0.1458      0.1704      0.5167    ]
 [ 0.56880971  0.56992329  0.51903246 -0.32232517  0.48449602]
 [ 0.58972373 -0.27715001  0.68753831 -0.37460027 -1.83408369]]


 P = 
 [2, 3, 1, 0, 4]


 c = 
 [ 0.0987     -0.61054152 -1.69805726  2.35015812  2.59163195]
x[4] -1.4130390910442732
